In [1]:
# 테라스, 케라스 관련 라이브러리 가져오기
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [2]:
# 데이터 가공 관련 라이브러리 가져오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2, os

In [3]:
# 이미지 데이터 불러오기 및 전처리
# 데이터 경로 설정
img_dir = ["./RUL/Data/Bearing_SFFT_Image/", "./RUL/Data/Gear_SFFT_Image/"]

# 이미지 갯수, 이미지 규격 설정
image_num = 240
image_w = 28
image_h = 28

# 학습 데이터 및 레이블 저장을 위한 리스트 선언
data = []
label = []

for index, factor in enumerate(img_dir):
    # one-hot-encoding 방식으로 레이블 지정
    for i in range(image_num):
        temp_label = [0, 0]
        temp_label[index] = 1
        label.append(temp_label)
    # 이미지 경로에서 파일이름을 불러오고 크기설정
    for top, dir, f in os.walk(factor):
        for filename in f:
            img = cv2.imread(factor + filename)
            img = cv2.resize(img, None, fx=image_w/img.shape[1], fy=image_h/img.shape[0])
            data.append(img/256)

In [4]:
# 학습데이터와 테스트 데이터로 구분
def train_test_split(X_data, y_data, test_size):
    X_train = X_data[0:int(len(X_data)*(1-test_size))]
    X_test = X_data[int(len(X_data)*(1-test_size)):]
    y_train = y_data[0:int(len(y_data)*(1-test_size))]
    y_test = y_data[int(len(y_data)*(1-test_size)):]
    return X_train, X_test, y_train, y_test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, label, 0.2)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [6]:
# 모델 생성
model = keras.Sequential()

In [7]:
# 모델 신경망 층 구성
model.add(Conv2D(32, kernel_size=(3,3), padding="same", input_shape=X_train.shape[1:], activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation="softmax"))

In [8]:
# 모델 컴파일
model.compile(
    loss="binary_crossentropy",
    optimizer="Adam",
    metrics=["accuracy"]
    )

In [9]:
# 모델 학습
with tf.device('/device:GPU:1'):
    model.summary()
    hist = model.fit(X_train, y_train, batch_size=32, epochs=100, shuffle=True, callbacks=[EarlyStopping(monitor="val_loss", patience=10)])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0

UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-9-e3af4f5abaa6>:4) ]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-9-e3af4f5abaa6>:4) ]]
	 [[gradient_tape/sequential/dense/MatMul_1/_56]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_972]

Function call stack:
train_function -> train_function


In [10]:
# 모델 평가
with tf.device('/device:GPU:1'):
    loss, acc = model.evaluate(X_test, y_test)

UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-10-29dad728dca0>:3) ]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-10-29dad728dca0>:3) ]]
	 [[Mul/_22]]
0 successful operations.
0 derived errors ignored. [Op:__inference_test_function_1137]

Function call stack:
test_function -> test_function


In [11]:
# 오차율, 정확도 그래프
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoche")
plt.legend(["Train", "Test"], loc=0)
plt.show()

KeyError: 'val_loss'

In [12]:
# 테스트 이미지 데이터 불러오기 및 전처리
# 데이터 경로 설정
img_dir = ["./RUL/Data/GearES_SFFT_Image/", "./RUL/Data/BearingES_Image/"]

# 이미지 갯수, 이미지 규격 설정
image_num = 3
image_w = 28
image_h = 28

# 학습 데이터 및 레이블 저장을 위한 리스트 선언
test_data = []
test_label = []

for index, factor in enumerate(img_dir):
    # one-hot-encoding 방식으로 레이블 지정
    for i in range(image_num):
        temp_label = [0, 0]
        temp_label[index] = 1
        test_label.append(temp_label)
    # 이미지 경로에서 파일이름을 불러오고 크기설정
    for top, dir, f in os.walk(factor):
        for filename in f:
            img = cv2.imread(factor + filename)
            img = cv2.resize(img, None, fx=image_w/img.shape[1], fy=image_h/img.shape[0])
            test_data.append(img/256)

In [13]:
# 예측
test_data = np.array(test_data)
try:
    with tf.device('/device:GPU:1'):
        predict = model.predict_classes(test_data)
except RuntimeError as e:
    print(e)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [14]:
with tf.device('/device:GPU:1'):
        predict_t = model.predict(test_data)

In [15]:
predict_t

array([[1.0000000e+00, 1.1520256e-22],
       [1.0000000e+00, 2.4222741e-18],
       [1.0000000e+00, 3.8989223e-27],
       [1.0000000e+00, 8.6914937e-17]], dtype=float32)

In [16]:
predict

array([0, 0, 0, 0])